In [1]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_252
Branch HEAD
Compiled by user centos on 2020-02-02T20:01:52Z
Revision cee4ecbb16917fa85f02c635925e2687400aa56b
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


In [10]:
!zcat /data/All_Amazon_Review.json.gz | head -n 1

{"overall": 1.0, "verified": false, "reviewTime": "12 11, 2015", "reviewerID": "A27BTSGLXK2C5K", "asin": "B017O9P72A", "reviewerName": "Jacob M. Wessler", "reviewText": "Alexa is not able to control my lights. If I ask her to tell me what LIFX can do, she will give me an example with one of my group names. If I use that exact same group name in a new request, she'll await that she doesn't recognize the name. This skill is VERY buggy and has not yet worked for me. I even rest Alexa, uninstalled LIFX, and set everything up again.", "summary": "VERY Buggy, doesn't work.", "unixReviewTime": 1449792000}
zcat: write error: Broken pipe
zcat: write: Broken pipe


In [11]:
!hdfs dfs -D dfs.replication=1 -put /data/All_Amazon_Review.json.gz /

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2025-04-25 23:23:46,390 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
!hdfs dfs -ls /All_Amazon_Review.json.gz


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2025-04-25 23:55:22,748 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   1 root supergroup 36001023911 2025-04-25 23:53 /All_Amazon_Review.json.gz


In [100]:
import time

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, from_unixtime, year, row_number
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .master('spark://master:7077') \
    .appName('PySpark') \
    .getOrCreate()

In [31]:
df = spark.read.json("hdfs://master:9000/All_Amazon_Review.json.gz")
df.write.mode("overwrite").orc("hdfs://master:9000/amazon_reviews_orc")

[Stage 1:>    (0 + 1) / 1][Stage 2:>    (0 + 1) / 1][Stage 3:>    (0 + 1) / 1]  

In [35]:
df_orc = spark.read.format("orc").load("/amazon_reviews_orc")

In [36]:
df_orc.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color Name:: string (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Flavor:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Package Quantity:: string (nullable = true)
 |    |-- Package Type:: string (nullable = true)
 |    |-- Scent Name:: string (nullable = true)
 |    |-- Size Name:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |    |-- Style Name:: string (nullable = true)
 |    |-- Style:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = 

In [45]:
df_orc_select = df_orc.select(
    "reviewerID", "verified", "reviewerName", 
    "asin", "reviewText", "summary",
    "overall", "unixReviewTime",
)

In [49]:
df_orc_select.show(5, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 reviewerID     | A27BTSGLXK2C5K                                                                                                                    

2025-04-26 00:36:45,352 WARN util.Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
[Stage 5:>                                                          (0 + 1) / 1]

##### PREWHERE

Получаем строки, где оценки товаров больше 3

In [94]:
start_time = time.time()
df_overall_filtered = df_orc_select.filter(col("overall") > 3)
df_result = df_overall_filtered.select("reviewerName", "reviewerID", "asin")

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.050 сек


In [95]:
df_result.limit(5).toPandas()

,reviewerName,reviewerID,asin
0,Greg,A27ZJ1NCBFP1HZ,B017O9P72A
1,Nello,A3TXR8GLKS19RE,B017O9P72A
2,Amazon Customer,A3KWZMO1A9TO3Z,B017O9P72A
3,Artyhow,A1QBG1TTQZGJNM,B017O9P72A
4,jarhead,A2LNJJWW2TLL00,B017O9P72A


##### INNER JOIN

Получаем дату и год из `unixReviewTime`, чтобы можно было быстро определить месяц, к примеру

In [101]:
start_time = time.time()

df_dates = df_orc_select.select(
    "reviewerID", "asin",
    from_unixtime("unixReviewTime").alias("review_date")
).withColumn("review_year", year("review_date"))

df_result_dates = df_orc_select.join(df_dates, on=["reviewerID", "asin"], how="inner")

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.034 сек


In [102]:
df_result_dates.limit(5).toPandas()

,reviewerID,asin,verified,reviewerName,reviewText,summary,overall,unixReviewTime,review_date,review_year
0,A27BTSGLXK2C5K,B017O9P72A,False,Jacob M. Wessler,Alexa is not able to control my lights. If I a...,"VERY Buggy, doesn't work.",1.0,1449792000,2015-12-11 00:00:00,2015
1,A27ZJ1NCBFP1HZ,B017O9P72A,False,Greg,"Alexa works great for me so far, but I'm also ...",So Far So Good,4.0,1449532800,2015-12-08 00:00:00,2015
2,ACCQIOZMFN4UK,B017O9P72A,False,Da-Gr8-1,Weak!!\n\nAlexa doesn't even recognize the nam...,Time waster,1.0,1449446400,2015-12-07 00:00:00,2015
3,A3KUPJ396OQF78,B017O9P72A,False,Larry Russlin,Can only control one of two bulbs from one of ...,Buggy,2.0,1449273600,2015-12-05 00:00:00,2015
4,A1U1RE1ZI19E1H,B017O9P72A,False,Rebekah,this worked great then randomly stopped. pleas...,stopped working,1.0,1517529600,2018-02-02 00:00:00,2018


##### FULL OUTER JOIN

Представим, что данные из разных источников и мы проверяем какие отстутствуют по `reviewerID`

In [85]:
df_left = df_orc_select.alias("left")
df_right = df_orc_select.alias("right")

In [88]:
start_time = time.time()

df_result = df_left.join(df_right, df_left.asin == df_right.asin, how="outer")
df_null_reviewerID = df_result.filter(col("left.reviewerID").isNull() | col("right.reviewerID").isNull())

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.025 сек


In [89]:
df_null_reviewerID.limit(5).toPandas()

,reviewerID,verified,reviewerName,asin,reviewText,summary,overall,unixReviewTime,reviewerID,verified,reviewerName,asin,reviewText,summary,overall,unixReviewTime


##### OVER QUALIFY

Получаем по одной строке с самым ранним временем ревью для каждого товара (asin) 

In [98]:
start_time = time.time()

window_spec = Window.partitionBy("asin").orderBy("unixReviewTime")
df_result = df_orc_select.withColumn("row_num", row_number().over(window_spec))
df_filtered = df_result.filter(col("row_num") == 1)

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.059 сек


In [99]:
df_filtered.limit(5).toPandas()

,reviewerID,verified,reviewerName,asin,reviewText,summary,overall,unixReviewTime,row_num
0,A9HQWY045DWAW,True,Margaret A. Westlake,B0007OC0A2,I had the worst trouble with blisters. No amo...,No more blisters and no more athletes foot,5.0,1165276800,1
1,AWD98R100TI8F,False,Robert Marotti,B000FEF1V4,"I have owned two previous ""accu-vac"" models. ...",A Vacuum Trimmer that actually VACUUMS!!,4.0,1160524800,1
2,A133FDJ7XNO7XJ,False,Cindy M. Mason,B000IQU2QS,Pre de Provence Green Tea Soap was recommended...,VERRY VERY GOOD,5.0,1200787200,1
3,A1DUQAIPY2G044,True,Justin,B001EHLSQS,I love this case and the car charger that has ...,Awesome case and charger,5.0,1260662400,1
4,ASNF7IPPLHON8,True,Savannah Dean,B0032RNS8E,I emailed the seller two week ago and was told...,I never received this item or a response from ...,1.0,1266192000,1


##### GROUP BY HAVING

Получим среднюю оценку за каждый год

In [106]:
start_time = time.time()

df_grouped = df_result_dates.groupBy("review_year").agg(
    F.avg("overall").alias("avg_overall")
)

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.015 сек


In [108]:
df_grouped.limit(5).toPandas().sort_values('review_year')

,review_year,avg_overall
0,2003,4.260870
4,2006,3.903585
1,2007,4.067637
3,2015,4.221330
2,2018,3.701237


##### GROUPING SETS

Получим агрегацию по множеству полей

In [126]:
start_time = time.time()

df_asin_grouped = df_orc_select.groupBy("asin").agg(
    F.avg("overall").alias("avg_overall")
).withColumn("reviewerID", F.lit(None))

df_reviewer_grouped = df_orc_select.groupBy("reviewerID").agg(
    F.avg("overall").alias("avg_overall")
).withColumn("asin", F.lit(None))

df_asin_reviewer_grouped = df_orc_select.groupBy("asin", "reviewerID").agg(
    F.avg("overall").alias("avg_overall")
)

df_grouping_sets = df_asin_grouped.union(df_reviewer_grouped).union(df_asin_reviewer_grouped)

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.113 сек


In [127]:
df_grouping_sets.limit(5).toPandas()

,asin,avg_overall,reviewerID
0,B01ELVUYCW,2.9607843137254903,None
1,B01G84HZKY,3.0,None
2,B01EWMV8XY,2.0,None
3,B01HBWUHVG,1.5,None
4,B003YUMRYK,4.375,None


##### EXCEPT

Просто хотим все строки, где оценка не 5, а про фильтрацию забыли

In [128]:
start_time = time.time()

df_orc_select_2 = df_orc_select.filter(col("overall") == 5.0) # здесь не считается
df_result = df_orc_select.exceptAll(df_orc_select_2)

end_time = time.time()
print(f"{end_time - start_time:.3f} сек")

0.049 сек


In [129]:
df_result.limit(5).toPandas()

,reviewerID,verified,reviewerName,asin,reviewText,summary,overall,unixReviewTime
0,A1X68G8AGFFEGC,False,R. Maday,B017OBE2NI,"Well, that was a disappointment. Created the a...",Disappointed,1.0,1482537600
1,A254VNC9HM5AQN,False,Ryan Houseman,B017OBYNZU,Needs to ask more questions. Answers are usual...,Almost there...,2.0,1449360000
2,A1TY81GW9CXZ4I,False,Renee,B017OBSCOS,She couldn't understand half of the words I said.,Sucks,1.0,1513036800
3,AML6X1TV7J2E3,False,Valerie T.,B017OBWZU0,This app malfunctions every 30seconds... Waist...,Terrible app,1.0,1453161600
4,A3ITH03ZCKIXD,False,D. Mehrten,B017OBSCOS,Her vocabulary is SO limited. She failed to re...,"Back to school, girl!",3.0,1483660800
